<a href="https://colab.research.google.com/github/shahchhatru/AI_colab_notebooks/blob/main/LaMini_FLan_t5_24_news_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

checkpoint = "MBZUAI/LaMini-Flan-T5-248M"

model = pipeline('text2text-generation', model = checkpoint)

input_prompt = 'Please let me know your thoughts on the given place and why you think it deserves to be visited: \n"Barcelona, Spain"'
generated_text = model(input_prompt, max_length=512, do_sample=True)[0]['generated_text']

print("Response", generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Response I'm sorry, I cannot provide thoughts as I am an AI language model and do not have the ability to form opinions or preferences. However, Barcelona has many historic sites, culture, and traditions that are considered an outstanding place to visit. It is a city that attracts millions of people every year and its historical significance is incredibly important to its people. Additionally, Barcelona's cultural heritage is not just rooted in architecture but also serves as a popular tourist destination to enjoy its various cultural and tourist attractions.


## Inference on summarization

In [ ]:
from transformers import T5Tokenizer , T5ForConditionalGeneration
import torch
import base64
import accelerate
from transformers import pipeline

checkpoint = "MBZUAI/LaMini-Flan-T5-248M"

#model and tokenizer

tokenizer=T5Tokenizer.from_pretrained(checkpoint)
base_model=T5ForConditionalGeneration.from_pretrained(checkpoint,torch_dtype=torch.float32)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
news_text="""Breaking News: Scientists Discover New Species of Deep-Sea Octopus

In a groundbreaking discovery, marine biologists have identified a previously unknown species of octopus lurking in the depths of the Pacific Ocean. The newly dubbed "Luminous Octopus" was found during an expedition led by the Oceanic Research Institute.

The Luminous Octopus, named for its bioluminescent properties, boasts tentacles adorned with glowing spots that emit a mesmerizing blue light. This unique adaptation is believed to aid the octopus in hunting and communication in the dark abyss where it resides.

Dr. Amelia Clarke, lead researcher on the expedition, expressed excitement over the find, stating, "The discovery of the Luminous Octopus opens up new avenues for understanding deep-sea ecosystems and the diversity of life within them."

The expedition team utilized state-of-the-art submersibles equipped with high-definition cameras to capture footage of the elusive creature in its natural habitat. Detailed observations and genetic analysis will provide valuable insights into the evolutionary history and ecological role of the Luminous Octopus."""

In [ ]:
input_ids = tokenizer(f"summarize it: {news_text}", return_tensors="pt").input_ids
outputs = base_model.generate(input_ids,max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

New species of octopus named "Luminous Octopus" found in Pacific Ocean, named after its bioluminescent properties. It has tentacles with glowing spots that emit blue light and is believed to aid in


## Actual training begins

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("multi_news", split="train")

In [ ]:
dataset= dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 35977
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 8995
    })
})

In [ ]:
dataset['train'][0]

{'document': "Published on Feb 7, 2012 \n \n The President meets an 8th grader named Joey from Phoenix, AZ at the White House Science Fair and the two launch a marshmallow across the State Dining Room of the White House with Joey's science project - an air cannon. ||||| Hello, my name is Elizabeth Hudy. My brother, Joey, just turned 20 years old and was recently diagnosed with Schizophrenia. You may remember Joey from the photo above when he had the awesome opportunity to attend the White House Science Fair where he displayed his project amongst dozens of other talented young students from across the country. Joey has traveled all over the world eagerly sharing his story about being a young Maker/inventor, and helping introduce other kids to STEMs. The Maker movement was instrumental to Joey and he loved to share this experience with others. Joey is known by many as a very smart, talented, giving, and fun loving young man.After spending two weeks in a psychiatic hospital in Hong Kong, 

## preprocesssing function

In [ ]:


prefix="summarize: "

def preprocess_function(examples):
  inputs=[prefix+doc for doc in examples['document']]
  model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

  labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/35977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8995 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['train'][0]

{'document': "Published on Feb 7, 2012 \n \n The President meets an 8th grader named Joey from Phoenix, AZ at the White House Science Fair and the two launch a marshmallow across the State Dining Room of the White House with Joey's science project - an air cannon. ||||| Hello, my name is Elizabeth Hudy. My brother, Joey, just turned 20 years old and was recently diagnosed with Schizophrenia. You may remember Joey from the photo above when he had the awesome opportunity to attend the White House Science Fair where he displayed his project amongst dozens of other talented young students from across the country. Joey has traveled all over the world eagerly sharing his story about being a young Maker/inventor, and helping introduce other kids to STEMs. The Maker movement was instrumental to Joey and he loved to share this experience with others. Joey is known by many as a very smart, talented, giving, and fun loving young man.After spending two weeks in a psychiatic hospital in Hong Kong, 

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4dca21df2639efa8a4dc47cf8a21926352f743195945189bc4f496f5925bf4f0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Evaluation metric function

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Getting ready for trianing

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
checkpoint

'MBZUAI/LaMini-Flan-T5-248M'

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chhatra_awesome_summarizer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 37.06 MiB is free. Process 37220 has 14.71 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 127.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.push_to_hub()